In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import SGD
import numpy as np
import os 

In [39]:
def get_jpg_path(path='birds\\train\\'):
    file_dir='birds\\train\\'
    path=[]
    clazz=[]
    for root, dirs, files in os.walk(file_dir): 
        if len(dirs)==0:
            cls=root[-3:]
            cls=[int(cls)]*len(files)
            ph=[root+"\\"+p for p in files]
            path.extend(ph)
            clazz.extend(cls)
    return path,clazz
path,clazz=get_jpg_path('birds\\train\\')

In [3]:
def onehot_label(clazz):
    index=0
    is_insight=set()
    hot=[0]*90
    onehot=[]
    for i in clazz:
        if i not in is_insight:
            is_insight.add(i)
            hot=[0]*90
            hot[index]=1
            index+=1
        onehot.append(hot)
    onehot=np.asarray(onehot)
    return onehot

In [4]:
onehot=onehot_label(clazz)
onehot.shape

(3830, 90)

In [32]:
def reszie_jpg(path):
    imgs=[]
    for p in path:
        im=Image.open(p)
        im=im.resize((224,224))
        imgs.append(np.asarray(im))
    imgs=np.asarray(imgs)
    return imgs

In [7]:
imgs=reszie_jpg(path)
imgs.shape

(3830, 224, 224, 3)

In [8]:
model = Sequential()
# BLOCK 1
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv1', input_shape = (224, 224, 3)))   
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv2'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block1_pool'))
 
# BLOCK2
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv1'))   
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv2'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block2_pool'))
 
# BLOCK3
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv1'))   
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv2'))
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv3'))
model.add(MaxPooling2D(pool_size = (4, 4), strides = (4, 4), name = 'block3_pool'))
 
# # BLOCK4
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv1'))   
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv3'))
# model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block4_pool'))
 
# # BLOCK5
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv1'))   
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv2'))
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv3'))
# model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), name = 'block5_pool'))
 
model.add(Flatten())
model.add(Dense(1024, activation = 'relu', name = 'fc1'))
model.add(Dropout(rate=0.5))
model.add(Dense(1024, activation = 'relu', name = 'fc2'))
model.add(Dropout(rate=0.5))
model.add(Dense(90, activation = 'softmax', name = 'prediction'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 14, 14, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 14, 14, 256)       295168    
__________

In [10]:
model.compile(optimizer=SGD(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
model.fit(imgs, onehot, epochs=1)

ValueError: Error when checking input: expected block1_conv1_input to have 4 dimensions, but got array with shape (34, 2, 224, 224, 3)

In [27]:
file_dir='birds\\train\\006'
path=[]
clazz=[]
for root, dirs, files in os.walk(file_dir): 
    if len(dirs)==0:
        cls=root[-3:]
        cls=[int(cls)]*len(files)
        ph=[root+"\\"+p for p in files]
        path.extend(ph)
        clazz.extend(cls)
print(len(path),len(clazz))

34 34


In [33]:
tests=reszie_jpg(path)
tests.shape

(34, 224, 224, 3)

In [34]:
rs=model.predict(tests)
rs

array([[0.00907632, 0.00741552, 0.01046429, 0.01604019, 0.00772488,
        0.00986239, 0.00775969, 0.01133312, 0.01259232, 0.00962495,
        0.01450036, 0.00971199, 0.01200068, 0.01988345, 0.0080014 ,
        0.00727588, 0.01542684, 0.01753281, 0.01599701, 0.00870604,
        0.0098861 , 0.01101942, 0.01026174, 0.01492045, 0.00932213,
        0.00893191, 0.00976159, 0.0074591 , 0.00846729, 0.01591012,
        0.01230937, 0.01033103, 0.01011078, 0.00763669, 0.00681461,
        0.011588  , 0.00981124, 0.01501656, 0.00730159, 0.00817472,
        0.01388377, 0.01115777, 0.00977756, 0.02060796, 0.01185687,
        0.01067476, 0.01011234, 0.00956468, 0.01920325, 0.01382851,
        0.01154592, 0.01187421, 0.00952453, 0.01243843, 0.0117469 ,
        0.00581912, 0.01094604, 0.01524426, 0.01499108, 0.01468741,
        0.01118314, 0.01178624, 0.00731886, 0.01251753, 0.01116586,
        0.00995525, 0.01079569, 0.01139764, 0.00633634, 0.0137929 ,
        0.01061052, 0.01009719, 0.01616655, 0.00

In [37]:
for i in rs:
    print(i.argmax())

43
13
16
18
16
16
10
16
18
29
18
17
7
7
16
3
18
16
43
16
29
13
16
3
3
16
16
18
18
29
16
13
16
18
